### Android Studio 설치
https://developer.android.com/studio 

python이 아님.. java / Kotlin 사용


### New project (새 프로젝트)

No activity

Name: 프로젝트 이름

Language: Java

Minimum SDK 지원되는 최소 버전 Android 7.0



#### 1-2 Gradle Scripts > build.gradle(Project)
```
buildscript {
    repositories {
        google()
        mavenCentral()
    }
    dependencies {
        classpath "com.android.tools.build:gradle:4.0.2"
    }
}
allprojects {
    repositories {
        google()
        mavenCentral()
        maven { url 'https://jitpack.io' }
    }
}
task clean(type: Delete) {
    delete rootProject.buildDir
}

```

#### 1-3 Gradle Scripts > build.gradle(Module)
```
plugins {
    id 'com.android.application'
}
android {
    compileSdkVersion 30
    buildToolsVersion "30.0.2"
    defaultConfig {
        applicationId "com.example.regressor"
        minSdkVersion 23
        targetSdkVersion 30
        versionCode 1
        versionName "1.0"
        testInstrumentationRunner "androidx.test.runner.AndroidJUnitRunner"
    }
    buildTypes {
        release {
            minifyEnabled false
            proguardFiles getDefaultProguardFile('proguard-android-optimize.txt'), 'proguard-rules.pro'
        }
    }
    compileOptions {
        sourceCompatibility JavaVersion.VERSION_1_8
        targetCompatibility JavaVersion.VERSION_1_8
    }
    aaptOptions {
        noCompress 'tflite'
    }
}
dependencies {
    implementation 'androidx.appcompat:appcompat:1.2.0'
    implementation 'com.google.android.material:material:1.1.0'
    implementation 'androidx.constraintlayout:constraintlayout:2.0.2'
    testImplementation 'junit:junit:4.+'
    androidTestImplementation 'androidx.test.ext:junit:1.1.2'
    androidTestImplementation 'androidx.test.espresso:espresso-core:3.3.0'
    implementation 'com.github.divyanshub024:AndroidDraw:v0.1'
    implementation 'org.tensorflow:tensorflow-lite:2.7.0'
    implementation 'org.tensorflow:tensorflow-lite-support:0.3.0'
}


```

#### 1-4 app > manifests > AndroidManifest.xml
```
<?xml version="1.0" encoding="utf-8"?>
<manifest xmlns:android="http://schemas.android.com/apk/res/android"
    package="com.example.regressor">
    <application
        android:allowBackup="true"
        android:icon="@mipmap/ic_launcher"
        android:label="@string/app_name"
        android:roundIcon="@mipmap/ic_launcher_round"
        android:supportsRtl="true"
        android:theme="@style/Theme.Regressor">
        <activity android:name="com.example.regressor.MainActivity">
            <intent-filter>
                <action android:name="android.intent.action.MAIN" />
                <category android:name="android.intent.category.LAUNCHER" />
            </intent-filter>
        </activity>
    </application>
</manifest>


```

#### 1-5 java > com.example.regressor > Classifier.java
```
package com.example.regressor;
import android.content.Context;
import android.content.res.AssetFileDescriptor;
import android.content.res.AssetManager;
import android.graphics.Bitmap;
import android.util.Log;
import android.util.Pair;
import org.tensorflow.lite.DataType;
import org.tensorflow.lite.Interpreter;
import org.tensorflow.lite.Tensor;
import org.tensorflow.lite.support.tensorbuffer.TensorBuffer;
import java.io.FileInputStream;
import java.io.IOException;
import java.nio.ByteBuffer;
import java.nio.ByteOrder;
import java.nio.channels.FileChannel;
public class Classifier {
    private static final String MODEL_NAME = "reg_model.tflite";
    Context context;
    Interpreter interpreter = null;
    int modelOutputClasses;
    public Classifier(Context context) {
        this.context = context;
    }
    public void init() throws IOException {
        ByteBuffer model = null;
        model = loadModelFile(MODEL_NAME);
        model.order(ByteOrder.nativeOrder());
        interpreter = new Interpreter(model);
        initModelShape();
    }
    private ByteBuffer loadModelFile(String modelName) throws IOException {
        AssetManager am = context.getAssets();
        AssetFileDescriptor afd = am.openFd(modelName);
        FileInputStream fis = new FileInputStream(afd.getFileDescriptor());
        FileChannel fc = fis.getChannel();
        long startOffset = afd.getStartOffset();
        long declaredLength = afd.getDeclaredLength();
        return fc.map(FileChannel.MapMode.READ_ONLY, startOffset, declaredLength);
    }
    private void initModelShape() {
        Tensor inputTensor = interpreter.getInputTensor(0);
        Tensor outputTensor = interpreter.getOutputTensor(0);
        int[] outputShape = outputTensor.shape();
        modelOutputClasses = outputShape[1];
    }
    public float classify(float input) {
        float[] inputData = {input};
        float[][] outputData = new float[1][1];
        interpreter.run(inputData, outputData);
        return outputData[0][0];
    }
    public void finish() {
        if (interpreter != null)
            interpreter.close();
    }
}

```

#### build.gradle(Project)
```build.gradle(Project)
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:orientation="vertical">
    <EditText
        android:id="@+id/editInput"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:ems="10"
        android:inputType="numberDecimal"
        android:text="" />
    <Button
        android:id="@+id/button"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:text="확인" />
    <TextView
        android:id="@+id/txtResult"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:textSize="20sp" />
</LinearLayout>

```

### JAVA

클래스: 파일

패키지: 디렉토리

문장 끝에 ; 세미콜론


### 
app 우클릭 new > model > assets :: assets 폴더 만들고 회귀모델(reg_model.tflite) 복+붙 


###  1-6. app > res > layout > activity_main.xml

```
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:orientation="vertical">

    <EditText
        android:id="@+id/editInput"
        android:layout_width="176dp"
        android:layout_height="30dp"
        android:ems="10"
        android:inputType="numberDecimal"
        android:text="" />

    <Button
        android:id="@+id/button"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:text="확인" />

    <TextView
        android:id="@+id/txtResult"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:textSize="20sp" />
</LinearLayout>

```

### 1-7 java > com.example.regressor > MainActivity.java
```
package com.example.regressor;

import androidx.appcompat.app.AppCompatActivity;

import android.os.Bundle;
import android.view.View;
import android.widget.Button;
import android.widget.EditText;
import android.widget.TextView;

public class MainActivity extends AppCompatActivity {
    // extends 상속  AppcompatActivity 상위클래스
    @Override  // 오버라이드: 기존 함수를 재정의
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        setContentView(R.layout.activity_main);
        // 앱의 화면 정의
        Button button = findViewById(R.id.button);
        EditText editInput = findViewById(R.id.editInput);
        TextView txtResult = findViewById(R.id.txtResult);
        button.setOnClickListener(new View.OnClickListener() {
            @Override
            public void onClick(View view) {
                String input = editInput.getText().toString();
                float value = Float.parseFloat(input);
                Classifier cls = new Classifier(MainActivity.this);
                try {
                    cls.init();
                    float result = cls.classify(value);
                    txtResult.setText(result + "");
                } catch (Exception e) {
                    e.printStackTrace();
                }
            }
        });
    }
}
```


### (Android Studio)


Device Manager

virtual

create device

release name / API level

'R' downlaod

Next 

가상장치 이름 AVD name 

(HAXM 설치)

github.com/intel/haxm/releases 

v7_8_0.zip 다운로드 & 설치

재부팅